目錄
0. 資料初步認識 Data Understanding
1. 資料擷取 Data Access
2. 資料合併 Data Mergeing
3. 離群值偵測 Outlier Detection
4. 缺失值填補 Data Imputation
5. 資料視覺化 Data Visualization
6. 特徵工程 Feature Engineering
透過特徵工程，著重資料重要的部分，本處筆資料是透過圖片的特徵工程而得到的
7. 特徵選擇 Feature Selecction
利用相關係數/模型/預測評估去選擇重要特徵

0. 資料說明 Data Drscribing(from kaggle)
This dataset comes from research by Semeion who was commissioned by the Centro Sviluppo Materiali (Italy) for this task.

The original aim of the research was to correctly classify the type of surface defects in stainless steel plates, with six types of possible defects (plus "other"). The Input vector was made up of 27 indicators that approximately describe the geometric shape of the defect and its outline.                                                                                                     URL: https://www.kaggle.com/datasets/uciml/faulty-steel-plates

In [4]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [5]:
df = pd.read_csv(r"C:\Users\sarah\My Python Stuff\Machine Learning\faults.csv")
df.head()

,X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,Maximum_of_Luminosity,...,Orientation_Index,Luminosity_Index,SigmoidOfAreas,Pastry,Z_Scratch,K_Scatch,Stains,Dirtiness,Bumps,Other_Faults
0,42,50,270900,270944,267,17,44,24220,76,108,...,0.8182,-0.2913,0.5822,1,0,0,0,0,0,0
1,645,651,2538079,2538108,108,10,30,11397,84,123,...,0.7931,-0.1756,0.2984,1,0,0,0,0,0,0
2,829,835,1553913,1553931,71,8,19,7972,99,125,...,0.6667,-0.1228,0.2150,1,0,0,0,0,0,0
3,853,860,369370,369415,176,13,45,18996,99,126,...,0.8444,-0.1568,0.5212,1,0,0,0,0,0,0
4,1289,1306,498078,498335,2409,60,260,246930,37,126,...,0.9338,-0.1992,1.0000,1,0,0,0,0,0,0


1. 資料擷取
1.1 column 及 row 的基本擷取
選取特定的行(column)

In [6]:
df['X_Minimum']

0         42
1        645
2        829
3        853
4       1289
        ... 
1936     249
1937     144
1938     145
1939     137
1940    1261
Name: X_Minimum, Length: 1941, dtype: int64

3. 離群值偵測 Outlier Detection
離群值代表對比其他資料下，某些資料的值與大部分資料產生極大差距，對後續分析及建模可能產生影響，需要注意資料的分配情形以利判斷。

常見離群值得出現有幾種
(1)由於資料輸入錯誤產生特殊值。
(2)特殊值可能代表特殊事件或重要的業務情境。例如機器故障、個人消費習慣，需要多方判斷。

以下利用兩種基本的方式介紹如何偵測離群值，本資料不太服從常態分配，導致 outlier 的狀況較多，如要保留這些被判斷是 outlier 的資料可以先進行資料轉換(取 log /開根號等等方式)讓資料較接近常態分配。

3.1 以標準化分數判斷
將資料轉成標準化分數或Z分數進行判斷，根據常態分配特性：

約 68% 的數據在平均值 ± 1 標準差內。
約 95% 的數據在平均值 ± 2 標準差內。
約 99.7% 的數據在平均值 ± 3 標準差內。

判斷離群值：Z 分數超過 ±3（或其他定義的閾值）的數據通常被視為離群值。

<!-- # 在進行轉換前，先將類別行變數移除再做標準化的動作，因為類別型變數通常是字串或編碼值，標準化這些欄位沒有意義（例如，將 0 和 1 作為數字進行計算可能導致誤差 -->


In [7]:
print(df.columns)

Index(['X_Minimum', 'X_Maximum', 'Y_Minimum', 'Y_Maximum', 'Pixels_Areas',
       'X_Perimeter', 'Y_Perimeter', 'Sum_of_Luminosity',
       'Minimum_of_Luminosity', 'Maximum_of_Luminosity', 'Length_of_Conveyer',
       'TypeOfSteel_A300', 'TypeOfSteel_A400', 'Steel_Plate_Thickness',
       'Edges_Index', 'Empty_Index', 'Square_Index', 'Outside_X_Index',
       'Edges_X_Index', 'Edges_Y_Index', 'Outside_Global_Index', 'LogOfAreas',
       'Log_X_Index', 'Log_Y_Index', 'Orientation_Index', 'Luminosity_Index',
       'SigmoidOfAreas', 'Pastry', 'Z_Scratch', 'K_Scatch', 'Stains',
       'Dirtiness', 'Bumps', 'Other_Faults'],
      dtype='object')


In [8]:
x = df.drop(["TypeOfSteel_A300", "TypeOfSteel_A400"],axis = 1)

from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
scaled_df = ss.fit_transform(x)
scaled_df = pd.DataFrame(scaled_df, columns = x.columns)

【用布林判斷是否為outlier⬇️】

(1)計算 DataFrame scaled_df 中每個數值的絕對值
(2)用布林值顯示每個數值的絕對值是否大於 3（Z 分數超過 ±3 的閾值）

In [9]:
abs_scaled_df = (abs(scaled_df)>3)
print(abs_scaled_df)

      X_Minimum  X_Maximum  Y_Minimum  Y_Maximum  Pixels_Areas  X_Perimeter  \
0         False      False      False      False         False        False   
1         False      False      False      False         False        False   
2         False      False      False      False         False        False   
3         False      False      False      False         False        False   
4         False      False      False      False         False        False   
...         ...        ...        ...        ...           ...          ...   
1936      False      False      False      False         False        False   
1937      False      False      False      False         False        False   
1938      False      False      False      False         False        False   
1939      False      False      False      False         False        False   
1940      False      False      False      False         False        False   

      Y_Perimeter  Sum_of_Luminosity  Minimum_of_Lu

【將「離群值檢測」彙總到「行的層級」，快速篩選包含離群值的數據行⬇️】

In [10]:
all_abs_scaled = abs_scaled_df.any(axis = 1)
print(all_abs_scaled)

0        True
1        True
2        True
3        True
4        True
        ...  
1936    False
1937    False
1938    False
1939    False
1940    False
Length: 1941, dtype: bool


In [13]:
outliar_index = all_abs_scaled[all_abs_scaled == True]
print(outliar_index)

0       True
1       True
2       True
3       True
4       True
        ... 
1846    True
1858    True
1863    True
1868    True
1876    True
Length: 671, dtype: bool


【找出離群值的索引、檢查特定行的離群值⬇️】

In [15]:
get_outliar_index = outliar_index.index
print(get_outliar_index)

Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
       ...
       1836, 1838, 1839, 1841, 1843, 1846, 1858, 1863, 1868, 1876],
      dtype='int64', length=671)


【將有 outlier 的資料刪除(原始資料行數1941)⬇️】

In [17]:
df_no_outliar = df.drop(outliar_index.index)
print(df_no_outliar.shape[0])

1270
